# Maximum Independent Set
Quantum Optimization Benchmark Library: https://doi.org/10.48550/arXiv.2504.03832

Hardware: Apple M1 Pro (8 cores, 16 GB memory)

## Benchmark

In [1]:
using Quicopt, LinearAlgebra, JSON, Printf
using CPUTime
include("../../src/utils.jl")
Base.show(io::IO, f::Float64) = @printf(io, "%1.5f", f)

In [2]:
PATH = "../../instances/max_independent_set/"
filename = "R_500_005_1.gph";

In [3]:
# load graph from file
graph = load_dimacs_graph_matrix(PATH * filename)

# system size
N = size(graph)[1]

500

In [4]:
# number of edges
size(findall(x -> x==1, UpperTriangular(graph)), 1)

6256

In [5]:
# Lagrange multiplier
λ = 1.0

# cost function
local_fields = -ones(N) ./ λ + 0.5 .* (ones(N)' * graph)';
couplings = -0.5 .* graph

tensor_problem = TensorProblem(local_fields, couplings);

### Fast Solution

In [6]:
# parameters
T = 2.0^7
tol = 1e-1;

In [7]:
# run once to force precompilation
mf_sol = solve(tensor_problem, T, atol=tol, rtol=tol)
sol_vec = sign.(mf_sol.u[end][3, :]);

In [8]:
# map from Ising to binary
bit_string = (1 .- sol_vec) ./ 2

# check violated clauses
num_violated_clauses = Int(0.5 * bit_string' * graph * bit_string)
num_violated_clauses |> println

# check max. indep. set
sum(bit_string) |> println

1
88.00000


In [9]:
false_idxs = findall(x -> x == 1., bit_string .* (graph * bit_string))
violated_bonds = Set([[idx, intersect(false_idxs, findall(x -> x == 1, graph[idx, :]))[1]] |> sort for idx in false_idxs])

# flip violated bonds
flip_idxs = [bond[1] for bond in violated_bonds]
bit_string[flip_idxs] .= 0.
bit_string' * graph * bit_string |> println
sum(bit_string) |> println

0.00000
87.00000


In [10]:
# get CPU time
@CPUtime begin
    mf_sol = solve(tensor_problem, T, atol=tol, rtol=tol)
    sol_vec = sign.(mf_sol.u[end][3, :])
    false_idxs = findall(x -> x == 1., bit_string .* (graph * bit_string))
    violated_bonds = Set([[idx, intersect(false_idxs, findall(x -> x == 1, graph[idx, :]))[1]] |> sort for idx in false_idxs])
    flip_idxs = [bond[1] for bond in violated_bonds]
    bit_string[flip_idxs] .= 0.
end;

elapsed CPU time: 0.39143 seconds


### Slow Solution

In [11]:
# parameters
T = 500.0
tol = 1e-4;

In [12]:
# run once to force precompilation
mf_sol = solve(tensor_problem, T, atol=tol, rtol=tol)
sol_vec = sign.(mf_sol.u[end][3, :]);

In [13]:
# map from Ising to binary
bit_string = (1 .- sol_vec) ./ 2

# check violated clauses
num_violated_clauses = Int(0.5 * bit_string' * graph * bit_string)
num_violated_clauses |> println

# check max. indep. set
sum(bit_string) |> println

5
94.00000


In [14]:
false_idxs = findall(x -> x == 1., bit_string .* (graph * bit_string))
violated_bonds = Set([[idx, intersect(false_idxs, findall(x -> x == 1, graph[idx, :]))[1]] |> sort for idx in false_idxs])

# flip violated bonds
flip_idxs = [bond[1] for bond in violated_bonds]
bit_string[flip_idxs] .= 0.
bit_string' * graph * bit_string |> println
sum(bit_string) |> println

0.00000
89.00000


In [15]:
OUT_PATH = "../../bitstrings/max_independent_set/"
open(OUT_PATH * filename[1:end-3] * "json", "w") do f
    print(f, bit_string .|> Int)
end

In [16]:
# get CPU time
@CPUtime begin
    mf_sol = solve(tensor_problem, T, atol=tol, rtol=tol)
    sol_vec = sign.(mf_sol.u[end][3, :])
    false_idxs = findall(x -> x == 1., bit_string .* (graph * bit_string))
    violated_bonds = Set([[idx, intersect(false_idxs, findall(x -> x == 1, graph[idx, :]))[1]] |> sort for idx in false_idxs])
    flip_idxs = [bond[1] for bond in violated_bonds]
    bit_string[flip_idxs] .= 0.
end;

elapsed CPU time: 1.95896 seconds
